# Responsável por tratar os dados pertinente a cliente

## Importando bibliotecas

In [17]:
import sys, os

# Carregando pacote que estao fora de /transform
path_raiz = os.path.abspath(os.path.join("..", ".."))
sys.path.append(path_raiz)

In [18]:
import pandas as pd
from src.utils import carregar_tabela_excel
from src.insert.insert_client import inserir_cliente
import json
import datetime

## Importando e tratando os dados

In [19]:
# Obter a data e hora atual
now = datetime.datetime.now()

# Formatar a data e hora como 'YYYYMMDD_HHMMSS'
timestamp = now.strftime("%Y%m%d_%H%M%S")
timestamp

'20250319_210118'

In [20]:
source_input = "../../data/baseTesteIXC_Transform.xlsx"
# source_output = f"../../src/inserted/cliente_insert_{timestamp}.xlsx"
source_output = f"../../src/inserted/cliente_insert_sucess.xlsx"

In [21]:
# Carregar o DataFrame com as colunas especificadas como strings
df = pd.read_excel(source_input, dtype=str).fillna("")

In [22]:
# Remover duplicatas
df = df.drop_duplicates(subset=["cpf_cnpj"], keep="first")

In [ ]:
# Lista das colunas que fazem sentido enviar para a API
colunas_para_api = [
    'id',
    'razaoSocial',
    'Email',
    'Celular',
    'cpf_cnpj',
    'rg_ie',
    'filial_id',
    'idCidade',
    'Endereco',
    'Complemento',
    'Numero',
    'Bairro',
    'Cep',
]

# Filtra o DataFrame para manter apenas essas colunas
df = df[colunas_para_api]


### Inserção teste

In [24]:
# inserir_cliente(
#     razao="Cliente de Teste 2",
#     cnpj_cpf="52.459.301/0001-81",
#     cep="57160-000",
#     endereco="Rua ABC",
#     numero="123",
#     bairro="Bairro XPTO",
#     filial_id="35",
#     cidade="1659",
#     uf="AL",
#     telefone_celular="82999999999",
#     email="teste@example.com",
# )

## Inserção do DF

In [25]:
# Limitando o DF
df = df.head(10)

In [26]:
# Log para verificação de campo vazio
print(df.isnull().sum())

razaoSocial    0
Email          0
Celular        0
cpf_cnpj       0
rg_ie          0
filial_id      0
idCidade       0
Endereco       0
Complemento    0
Numero         0
Bairro         0
Cep            0
dtype: int64


In [ ]:
# Crie uma cópia do DataFrame original para evitar o SettingWithCopyWarning
df = df.copy()

# Crie as colunas adicionais (inicialmente vazias)
df.loc[:, "idClienteIXC"] = None
df.loc[:, "statusInsercao"] = None
df.loc[:, "logRetorno"] = None

for idx, row in df.iterrows():
    # Mapeamos as colunas do DF para os parâmetros da função inserir_cliente.
    razao = row["razaoSocial"]
    email = row["Email"]
    celular = row["Celular"]
    cnpj_cpf = row["cpf_cnpj"]
    rg_ie = row["rg_ie"]
    filial = row["filial_id"]
    cidade = row["idCidade"] 
    endereco = row["Endereco"]
    complemento = row["Complemento"] if pd.notnull(row["Complemento"]) else ""
    numero = row["Numero"]
    bairro = row["Bairro"]
    cep = row["Cep"]
    idx = row["id"]
    
    # Chamamos a função inserir_cliente:
    response = inserir_cliente(
        razao=razao,
        cnpj_cpf=cnpj_cpf,
        cep=cep,
        endereco=endereco,
        numero=numero,
        complemento=complemento,
        bairro=bairro,
        cidade=cidade,
        filial_id=filial,
        telefone_celular=celular,
        email=email,
        ie_identidade=rg_ie,
        idx=idx
        # Se você quiser passar outros campos com valores-padrão, inclua aqui
        # Ex.: uf="AL" etc.
    )
    
    status_code = response.status_code
    content = response.content  # bytes

    # Fazemos parse do JSON:
    # b'{"type":"success","message":"Registro inserido com sucesso!","id":"117580", ...}'
    json_resp = json.loads(content.decode("utf-8", errors="replace"))

    if status_code == 200 and json_resp.get("type") == "success":
        # Sucesso: capturamos o "id"
        client_id = json_resp.get("id")  # Exemplo: "117580"
        
        df.at[idx, "idClienteIXC"] = client_id
        df.at[idx, "statusInsercao"] = "sucesso"
        # logRetorno pode ficar vazio ou None em caso de sucesso
        df.at[idx, "logRetorno"] = json_resp
    else:
        # Falha: salvamos o log completo
        df.at[idx, "idClienteIXC"] = None
        df.at[idx, "statusInsercao"] = "falha"
        # Guardamos o JSON completo (ou a chave "message", se preferir):
        df.at[idx, "logRetorno"] = json_resp

200 OK
b'{"type":"success","message":"Registro inserido com sucesso!","id":"117693","atualiza_campos":[{"tipo":"r","campo":"ativo","valor":"S"},{"tipo":"i","campo":"data_cadastro","valor":"19\\/03\\/2025"},{"tipo":"i","campo":"filial_id","valor":"35"},{"tipo":"i","campo":"latitude","valor":""},{"tipo":"i","campo":"longitude","valor":""},{"tipo":"i","campo":"id_conta","valor":"919704"},{"tipo":"d","campo":"crm_data_vencemos","valor":""},{"tipo":"r","campo":"convert_cliente_forn","valor":""},{"tipo":"d","campo":"crm_data_perdemos","valor":""},{"tipo":"i","campo":"crm_data_sem_viabilidade","valor":""},{"tipo":"i","campo":"crm_data_sem_porta_disponivel","valor":""},{"tipo":"i","campo":"crm_data_abortamos","valor":""},{"tipo":"i","campo":"crm_data_negociando","valor":""},{"tipo":"i","campo":"crm_data_apresentando","valor":""},{"tipo":"i","campo":"crm_data_sondagem","valor":""},{"tipo":"i","campo":"crm_data_novo","valor":""}]}'


### Retorno

In [28]:
df.to_excel(source_output)

In [29]:
df

,razaoSocial,Email,Celular,cpf_cnpj,rg_ie,filial_id,idCidade,Endereco,Complemento,Numero,Bairro,Cep,idClienteIXC,statusInsercao,logRetorno
0,Abelanio Veríssimo Peixoto,teste@gmail.com,99999999999,014.846.390-81,Isento,35,1659,Vila São Sebastião,Próximo ao cacimbão,0,Centro,57830-000,117693,sucesso,"{'type': 'success', 'message': 'Registro inser..."
